In [ ]:
#!/usr/bin/env python3
# -*- coding: utf-8 -*-
"""
Created on Wed May 29 15:40:50 2019

@author: soumyadeep
"""

#****************************************
# ORBITAL PROJECTED BAND STRUCTURE      *
#****************************************

#**********************************************************************
#               PART-1                                                *
# MAKING OF DATA FILE FOR PROJECTIONS                                 *
#**********************************************************************

import matplotlib.mlab as mlab
import matplotlib.pyplot as plt
import matplotlib.gridspec as gs
from matplotlib import cm
from matplotlib.mlab import griddata
from matplotlib import rc
import matplotlib.ticker as ticker
import matplotlib.lines
import csv
from collections import OrderedDict
import xml.etree.ElementTree as ET
import numpy as np
import itertools
import sys, os
import glob   # for stacking csv files
from matplotlib.ticker import FormatStrFormatter
from matplotlib.ticker import (AutoMinorLocator, MultipleLocator)
from mpl_toolkits.axes_grid1.inset_locator import zoomed_inset_axes
from mpl_toolkits.axes_grid1.inset_locator import mark_inset
import matplotlib.patches as patches
from matplotlib import rc,rcParams
import time
start_time = time.time()



NAME = 'CaKFe4As4'
EXAMPLE_DIR = '/media/soumyadeep/SOUMYA2/Python/1144/CaK/GGA/XGYGXG'
SAVE_DIR='/media/soumyadeep/SOUMYA2/Python/1144/CaK/GGA/XGYGXG'

# MATERIAL SPECIFIC INPUT
sspin = 1 #selected spin component.
nproj=32  #number of projections 
# decrease wave function number by one unit e.g. for wfc10 write wfc9
wf_number=[8,9,10,11,12,21,22,23,24,25,34,35,36,37,38,47,48,49,50,51,53,54,55,57,58,59,61,62,63,65,66,67]    
Efermi=9.7277    # Fermi energy in eV

# TO REMOVE OLDER FILES, OTHERWISE COMMENT OUT
#os.remove(EXAMPLE_DIR+'/data.csv')
#os.remove(EXAMPLE_DIR+'/energy.csv')
#os.remove(EXAMPLE_DIR+'/k_points.csv')
#os.remove(EXAMPLE_DIR+'/projections.csv')

# TARGET DIRECTORY FOR SAVING PROJECTION FILES
os.mkdir(EXAMPLE_DIR+'/proj_info')

# PARSING .xml FILE FROM EXAMPLE DIR
tree = ET.parse(EXAMPLE_DIR+'/atomic_proj.xml')
root = tree.getroot()


# READING OF .xml FILE
header = root.find('HEADER')
nbnd  = int(header.find('NUMBER_OF_BANDS').text.strip())
nkpts = int(header.find('NUMBER_OF_K-POINTS').text.strip())
natwfc= int(header.find('NUMBER_OF_ATOMIC_WFC').text.strip())
nspin = int(header.find('NUMBER_OF_SPIN_COMPONENTS').text.strip())

print(' ============================================================')
print('|  ORBITAL PROJECTED BAND-STRUCTURE PROGRAM @ by-SOUMYADEEP  |')
print(' ============================================================')

print('system information')
print('Total no of bands=',nbnd)
print('Total no of kpoints=',nkpts)
print('Total no of atomic projections=',natwfc)
print('Total spin=',nspin)

# START OF PROGRAM
print('PART-1 running ........')

d = root.find('K-POINTS')
kpts = np.array(d.text.strip().split(),dtype=np.float)
kpts = kpts.reshape([nkpts,3])
for i in range(len(kpts)):
    a=kpts[i][0]       # kx
    b=kpts[i][1]       # ky
    c=kpts[i][2]       # kz
    if i==0:
        k=0.0
    else:
        k+=np.sqrt((kpts[i][0]-kpts[i-1][0])**2+(kpts[i][1]-kpts[i-1][1])**2+(kpts[i][2]-kpts[i-1][2])**2)
    with open(EXAMPLE_DIR+"/k_points.csv","a") as output_file:
      output_fileWriter=csv.writer(output_file)
      output_fileWriter.writerow([a,b,c,k])
    output_file.close()

print('K-POINT CARD GENERATED')




d = root.find('WEIGHT_OF_K-POINTS')
wgth = np.array(d.text.strip().split(),dtype=np.float)


eigs = root.find('EIGENVALUES')
es = []
for i in range(nkpts):
    # set estr, the string for parsing eigenvalues
    if nspin > 1:
        estr = 'EIG.'+str(sspin)
    else:
        estr = 'EIG'
        
    e = np.array(eigs.find('K-POINT.'+str(i+1)).find(estr).text.strip().split(),dtype=np.float)
    es.append(e)
for i in range(nkpts):
    for j in range(nbnd):
        #print(es[i][j])
        with open(EXAMPLE_DIR+"/energy.csv","a") as output_file:
            output_fileWriter=csv.writer(output_file)
            output_fileWriter.writerow([es[i][j]])
        output_file.close()
    
print('EIGENVALUE CARD GENERATED')


p = root.find('PROJECTIONS')

wfcs=[]
for nk in range(nkpts):
    k = p.find('K-POINT.'+str(nk+1))
    wfc = np.zeros([natwfc,nbnd],dtype=np.complex)
    #print(wfc)
    
    if nspin > 1:
        k = k.find('SPIN.'+str(sspin))
    
    for a in range(natwfc):
        d = k.find('ATMWFC.'+str(a+1)).text.strip().split('\n')
        #print(d)
    
        for i in range(nbnd):
            wfc[a,i]=np.complex(*[float(x) for x in d[i].split(',')])
    wfcs.append(wfc)


rwfcs = np.swapaxes(wfcs,1,2)
#print(rwfcs[99][40][60])

#wfcs[:,[1, 2]] = wfcs[:,[2, 1]]
#print(rwfcs)


for i in range(nkpts):
    for j in range(nbnd):
        for k in range(natwfc): 
            proj=rwfcs[i][j][k].flat[0]       
            if k in (wf_number):
            #if (24 <= k <= 28 or 37 <= k <= 41):
            #if (2 <= k <= 7):
                #print(k)
                with open(EXAMPLE_DIR+"/projections.csv","a") as output_file:
                    output_fileWriter=csv.writer(output_file)
                    output_fileWriter.writerow([i,j,k,proj.real,proj.imag,abs(proj)])
                output_file.close()

print('PROJECTION CARD GENERATED')


kx,ky,kz,k = np.loadtxt(EXAMPLE_DIR+'/k_points.csv', delimiter=',', unpack=True)
E = np.loadtxt(EXAMPLE_DIR+'/energy.csv', delimiter=',', unpack=True)
ii,jj,kk,Pr,Pi,Pa = np.loadtxt(EXAMPLE_DIR+'/projections.csv', delimiter=',', unpack=True)


#print(len(k),len(E),len(Pa))
for x in range(nkpts):
    for y in range(nbnd):
        y+=nbnd*x
        for z in range(nproj):
            z+=nproj*y
            #print(x,y,z)
            with open(EXAMPLE_DIR+"/data.csv","a") as output_file:
                output_fileWriter=csv.writer(output_file)
                output_fileWriter.writerow([k[x],E[y],Pa[z]])
            output_file.close()

print('DATA CARD GENERATED')            
print('PART-1 completed ........')
           
print('time to complete PART-1=%.4s seconds' % (time.time() - start_time))

#**********************************************************************
#               PART-2                                                *
# SEPARATE PROJECTIONS FOR SCATTER PLOT                               *
# change if wave function number is changed                           *
#**********************************************************************
print('PART-2 running ........')

for band_number in range(0,nbnd,1):
    #print(band_number)
    k, E, Pa = np.loadtxt(EXAMPLE_DIR+'/data.csv', delimiter=',', unpack=True)
    E=(E*13.6)-Efermi
    for j in range(0,len(k),nbnd*nproj):   
        i=j+nproj*(band_number-1)   
        #print(k[i],E[i],Pa[i]+Pa[i+5])
        Ptot=Pa[i]+Pa[i+1]+Pa[i+2]+Pa[i+3]+Pa[i+4]+Pa[i+5]+Pa[i+6]+Pa[i+7]+Pa[i+8]+Pa[i+9]+Pa[i+10]+Pa[i+11]+Pa[i+12]+Pa[i+13]+Pa[i+14]+Pa[i+15]
        +Pa[i+16]+Pa[i+17]+Pa[i+18]+Pa[i+19]+Pa[i+20]+Pa[i+21]+Pa[i+22]+Pa[i+23]+Pa[i+24]+Pa[i+25]+Pa[i+26]+Pa[i+27]+Pa[i+28]+Pa[i+29]+Pa[i+30]+Pa[i+31]
        with open(EXAMPLE_DIR+"/proj_info/proj_" + str(band_number+1) + ".csv","a") as output_file:
            output_fileWriter=csv.writer(output_file) 
            output_fileWriter.writerow([k[i],E[i],
                                    (Pa[i]+Pa[i+5]+Pa[i+10]+Pa[i+15])/Ptot,
                                    (Pa[i+1]+Pa[i+6]+Pa[i+11]+Pa[i+16])/Ptot,
                                    (Pa[i+2]+Pa[i+7]+Pa[i+12]+Pa[i+17])/Ptot,
                                    (Pa[i+3]+Pa[i+8]+Pa[i+13]+Pa[i+18])/Ptot,
                                    (Pa[i+4]+Pa[i+9]+Pa[i+14]+Pa[i+19])/Ptot,
                                    (Pa[i+20]+Pa[i+23]+Pa[i+26]+Pa[i+29])/Ptot,
                                    (Pa[i+21]+Pa[i+24]+Pa[i+27]+Pa[i+30])/Ptot,
                                    (Pa[i+22]+Pa[i+25]+Pa[i+28]+Pa[i+31])/Ptot])
            output_fileWriter=csv.writer(output_file)

print('PLOTTING CARD GENERATED') 
print('PART-2 completed ........')

print('time to complete PART-2=%.4s seconds' % (time.time() - start_time))

#**********************************************************************
#               PART-3                                                *
# PLOTTING OF ORBITAL PROJECTED BAND STRCUTURE                        *
# change if wave function weight is changed                           *
#**********************************************************************
print('PART-3 running ........')
print('PLOTTING OPBS')

circle=0   #size of the circle
path = EXAMPLE_DIR+'/proj_info/proj_*.csv'

fig,ax = plt.subplots()
for fname in glob.glob(path):
    #print(fname)    
    k, E, d_z2, d_zx, d_zy, d_x2y2, d_xy, p_z, p_x, p_y = np.loadtxt(fname, delimiter=',', unpack=True)
    k = k/max(k)
    plt.plot(k, E, c='k',lw=0.5,alpha=0.4)
    
    p1=plt.scatter(k, E, s=(circle*d_z2)**2, c='magenta', marker='.',alpha=0.5,label='$3d_{z^2}$')
    p2=plt.scatter(k, E, s=(10*d_zx)**2, c='navy', marker='.',alpha=0.5,label='$3d_{xz}$')
    p3=plt.scatter(k, E, s=(10*d_zy)**2, c='r', marker='.',alpha=0.5,label='$3d_{yz}$')
    p4=plt.scatter(k, E, s=(circle*d_x2y2)**2, c='orange', marker='.',alpha=0.4,label='$3d_{x^2-y^2}$')
    p5=plt.scatter(k, E, s=(circle*d_xy)**2, c='green', marker='.',alpha=0.8,label='$3d_{xy}$')
    p6=plt.scatter(k, E, s=(circle*p_z)**2, c='cyan', marker='.',alpha=0.5,label='$4p_{z}$')
    #p7=plt.scatter(k, E, s=(circle*p_x)**2, c='dimgray', marker='.',alpha=0.5,label='$4p_{x}$')
    #p8=plt.scatter(k, E, s=(circle*p_y)**2, c='olive', marker='.',alpha=0.5,label='$4p_{y}$')
    
    
# AXES PROPERTIES
plt.ylim(-1.2,1.2)
#plt.yticks(np.arange(0, 1, step=0.2))
plt.xlim(min(k), max(k))
plt.axhline(y=0.0, color='k',ls='--',lw=1.0)
plt.axvline(x=0.2,color='k', ls='--',lw=1.0)
plt.axvline(x=0.4,color='k', ls='--',lw=1.0)
plt.axvline(x=0.6,color='k', ls='--',lw=1.0)
plt.axvline(x=0.8,color='k', ls='--',lw=1.0)
#plt.axvline(x=0.434,color='k', ls='--',lw=1.0)
#plt.axvline(x=0.565,color='k', ls='--',lw=1.0)
#plt.axvline(x=0.738,color='k', ls='--',lw=1.0)
#plt.axvline(x=0.868,color='k', ls='--',lw=1.0)
#plt.xticks([0.0,0.172,0.303,0.434,0.565,0.738,0.868,1.0], ['$\Gamma$', 'X', 'M', '$\Gamma$', 'Z', 'R', 'A', 'Z'],fontsize=18)
plt.xticks([0.0,0.2,0.4,0.6,0.8,1.0], [r'$\rm M_X$','$\Gamma$', r'$\rm M_Y$', '$\Gamma$', r'$\rm M_X$', '$\Gamma$'],fontsize=18)
plt.ylabel('Energy (eV)',fontsize=18,fontweight='bold')
plt.tick_params(axis='y', which='major', length=5.0, direction='out', bottom=False, top=False, left=True, right=False, labelsize = 12.0)
plt.tick_params(axis='y', which='minor', length=3.0, direction='out', bottom=False, top=False, left=True, right=False, labelsize = 12.0)
#ml = MultipleLocator(2)
ax.yaxis.set_major_locator(MultipleLocator(0.2))
ax.yaxis.set_major_formatter(FormatStrFormatter('%.1f'))
ax.yaxis.set_minor_locator(MultipleLocator(2))
plt.rcParams["font.weight"] = "bold"
plt.rcParams["axes.labelweight"] = "bold"

# WRITE Ef IN RIGHT SIDE
#plt.annotate(r'$\rm E_{F}$', xy=(0,0), xytext=(1.01,-0.05),fontsize=18,weight="bold")

# set up legends
lgnd=plt.legend((p2,p3), 
    (r'$\rm d_{xz}$',r'$\rm d_{yz}$'),
    loc='lower center', bbox_to_anchor=(0.5, 1.02), markerscale=6,
    fancybox=True, shadow=False, ncol=2 ,fontsize=12, borderaxespad=0.1) 

#lgnd=plt.legend((p1,p2,p3,p4,p5,p6), 
#    ('$3d_{z^2}$','$3d_{xz}$','$3d_{yz}$','$3d_{x^2-y^2}$','$3d_{xy}$','$4p_{z}$'),
#    loc='lower center', bbox_to_anchor=(0.5, 1.02), markerscale=6,
#    fancybox=True, shadow=False, ncol=3 ,fontsize=12, borderaxespad=0.1) 

#change the marker size manually for both lines
lgnd.legendHandles[0]._sizes = [1000]
lgnd.legendHandles[1]._sizes = [1000]
#lgnd.legendHandles[2]._sizes = [1000]
#lgnd.legendHandles[3]._sizes = [1000]
#lgnd.legendHandles[4]._sizes = [1000]
#lgnd.legendHandles[5]._sizes = [1000]
#lgnd.legendHandles[6]._sizes = [1000]
#lgnd.legendHandles[7]._sizes = [1000]

#==============================================================================       
# FOR ZOOMING A PARTICULAR PART OF PLOT-1
#==============================================================================
#from matplotlib.transforms import blended_transform_factory
#transform = blended_transform_factory(fig.transFigure, ax.transAxes)
axins1 = zoomed_inset_axes(ax, 6, loc=8, bbox_to_anchor=(0.9,-0.09,1,1.5), bbox_transform=ax.transAxes) # zoom-factor: 2.5, location: upper-left
for fname in glob.glob(path):
    k, E, d_z2, d_zx, d_zy, d_x2y2, d_xy, p_z, p_x, p_y = np.loadtxt(fname, delimiter=',', unpack=True)
    k = k/max(k)
    plt.plot(k, E, c='k',lw=1,alpha=0.4)
    p2=plt.scatter(k, E, s=(20*d_zx)**2, c='navy', marker='.',alpha=0.8,label='$3d_{xz}$')
    p3=plt.scatter(k, E, s=(20*d_zy)**2, c='r', marker='.',alpha=0.8,label='$3d_{yz}$')
    #for xy in zip(k, E):                                       
        #axins1.annotate('(%s, %.3f)' % xy, xy=xy, textcoords='data',fontsize=3)
#np1,=axins1.plot(k, E, c='navy',lw=0.5,alpha=1,label='dxy')                                      #left, bottom, width, height
#np2,=axins1.plot(k, E, c='r',lw=0.5,alpha=1,ls='--',label='dxz')
axins1.set_xlim(0.755,0.845) # apply the x-limits
axins1.set_ylim(-1.01,-0.79) # apply the y-limits
#axins1.yaxis.tick_right()
axins1.tick_params(axis='y', which='major', direction='inout', pad=0.5, labelsize = 6.0)
axins1.yaxis.set_major_locator(plt.MaxNLocator(5))
axins1.yaxis.set_major_formatter(FormatStrFormatter('%1d'))
axins1.set_xticks([])
axins1.set_yticks([])
axins1.axvline(x=0.8,color='k', ls='--',lw=1.0,alpha=0.5)
axins1.set_xticks([0.8])
axins1.set_xticklabels([r'$\rm M_X$'],fontsize=10)
mark_inset(ax, axins1, loc1=2, loc2=3, facecolor="none", alpha=1,lw=0.6, edgecolor='g',ls='--')

#==============================================================================       
# FOR ZOOMING A PARTICULAR PART OF PLOT-2
#==============================================================================
#from matplotlib.transforms import blended_transform_factory
#transform = blended_transform_factory(fig.transFigure, ax.transAxes)
axins2 = zoomed_inset_axes(ax, 6, loc=8, bbox_to_anchor=(0.9,0.53,1,1.5), bbox_transform=ax.transAxes) # zoom-factor: 2.5, location: upper-left
for fname in glob.glob(path):
    k, E, d_z2, d_zx, d_zy, d_x2y2, d_xy, p_z, p_x, p_y = np.loadtxt(fname, delimiter=',', unpack=True)
    k = k/max(k)
    plt.plot(k, E, c='k',lw=1,alpha=0.4)
    p2=plt.scatter(k, E, s=(20*d_zx)**2, c='navy', marker='.',alpha=0.8,label='$3d_{xz}$')
    p3=plt.scatter(k, E, s=(20*d_zy)**2, c='r', marker='.',alpha=0.8,label='$3d_{yz}$')
    #for xy in zip(k, E):                                       
        #axins2.annotate('(%s, %.3f)' % xy, xy=xy, textcoords='data',fontsize=3) 
#np1,=axins2.plot(k, E, c='navy',lw=0.5,alpha=1,label='dxy')                                      #left, bottom, width, height
#np2,=axins2.plot(k, E, c='r',lw=0.5,alpha=1,ls='--',label='dxz')
axins2.set_xlim(0.355,0.445) # apply the x-limits
axins2.set_ylim(-1.01,-0.79) # apply the y-limits
axins2.axvline(x=0.333,color='k', ls='--',lw=1)
axins2.tick_params(axis='y', which='major', direction='inout', pad=0.5, labelsize = 6.0)
axins2.yaxis.set_major_locator(plt.MaxNLocator(5))
axins2.yaxis.set_major_formatter(FormatStrFormatter('%1d'))
axins2.set_xticks([])
axins2.set_yticks([])
axins2.axvline(x=0.4,color='k', ls='--',lw=1.0,alpha=0.5)
axins2.set_xticks([0.4])
axins2.set_xticklabels([r'$\rm M_Y$'],fontsize=10)
axins2.xaxis.tick_top()
mark_inset(ax, axins2, loc1=2, loc2=3, facecolor="none", alpha=1,lw=0.6, edgecolor='g',ls='--')

#==============================================================================       
# FOR ZOOMING A PARTICULAR PART OF PLOT-3
#==============================================================================
#from matplotlib.transforms import blended_transform_factory
#transform = blended_transform_factory(fig.transFigure, ax.transAxes)
axins3 = zoomed_inset_axes(ax, 6, loc=8, bbox_to_anchor=(-1,0.69,1,1.5), bbox_transform=ax.transAxes) # zoom-factor: 2.5, location: upper-left
for fname in glob.glob(path):
    k, E, d_z2, d_zx, d_zy, d_x2y2, d_xy, p_z, p_x, p_y = np.loadtxt(fname, delimiter=',', unpack=True)
    k = k/max(k)
    plt.plot(k, E, c='k',lw=1,alpha=0.4)
    p2=plt.scatter(k, E, s=(20*d_zx)**2, c='navy', marker='.',alpha=0.8,label='$3d_{xz}$')
    p3=plt.scatter(k, E, s=(20*d_zy)**2, c='r', marker='.',alpha=0.8,label='$3d_{yz}$')
#np1,=axins3.plot(k, E, c='navy',lw=0.5,alpha=1,label='dxy')                                      #left, bottom, width, height
#np2,=axins3.plot(k, E, c='r',lw=0.5,alpha=1,ls='--',label='dxz')
axins3.set_xlim(0.55,0.65) # apply the x-limits
axins3.set_ylim(0.2,0.4) # apply the y-limits
#axins3.yaxis.tick_right()
axins3.tick_params(axis='y', which='major', direction='inout', pad=0.5, labelsize = 6.0)
axins3.yaxis.set_major_locator(plt.MaxNLocator(5))
axins3.yaxis.set_major_formatter(FormatStrFormatter('%1d'))
axins3.set_xticks([])
axins3.set_yticks([])
axins3.axvline(x=0.6,color='k', ls='--',lw=1.0,alpha=0.5)
axins3.set_xticks([0.6])
axins3.set_xticklabels(['$\Gamma$'],fontsize=12)
#axins3.xaxis.tick_top()
mark_inset(ax, axins3, loc1=4, loc2=1, facecolor='none', alpha=1,lw=0.6, edgecolor='g',ls='--')

# CREATE SOME SPACE IN FOUR SIDES OF PLOT
plt.subplots_adjust(top=0.83, left=0.33, right=0.72, bottom=0.1)  

# SAVING PART
#plt.tight_layout()
fig.set_size_inches(12,5.5)
plt.savefig('/media/soumyadeep/SOUMYA2/Python/test_fig.pdf', aspect='auto', dpi=4500, alpha=0.1)
#plt.savefig(SAVE_DIR+"/"+NAME+"_OPBS_ns_Y+G+X.pdf",bbox_inches='tight', dpi=4500)  
#plt.savefig(SAVE_DIR+"/"+NAME+"_OPBS_ns_Y+G+X.eps",bbox_inches='tight', dpi=4500)    
#plt.savefig(SAVE_DIR+"/"+NAME+"_OPBS_ns_Y+G+X.jpeg",bbox_inches='tight', dpi=1500)  
plt.show() 



print('PART-3 completed ........')
print('time to complete PART-3=%.4s seconds' % (time.time() - start_time))
print('ALL DONE')
